In [2]:
import pandas as pd
import nltk
import sklearn
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, SnowballStemmer
from utils_processor.processor import Processor
import re 
from sklearn.model_selection import train_test_split

processor_ = Processor()

# Processing Pipeline

In [3]:
df_20N = pd.read_csv("data/20N_unified.csv")

In [4]:
df_20N

,Unnamed: 0,category,email_id,from,subject,body
0,0,alt.atheism,49960,mathew <mathew@mantis.co.uk>,Alt.Atheism FAQ: Atheist Resources,Archive-name: atheism/resources Alt-atheism-ar...
1,1,alt.atheism,51060,mathew <mathew@mantis.co.uk>,Alt.Atheism FAQ: Introduction to Atheism,Archive-name: atheism/introduction Alt-atheism...
2,2,alt.atheism,51119,I3150101@dbstu1.rz.tu-bs.de (Benedikt Rosenau),Re: Gospel Dating,In article <65974@mimsy.umd.edu> mangoe@cs.umd...
3,3,alt.atheism,51120,mathew <mathew@mantis.co.uk>,Re: university violating separation of church/...,dmn@kepler.unh.edu (...until kings become phil...
4,4,alt.atheism,51121,strom@Watson.Ibm.Com (Rob Strom),"Re: [soc.motss, et al.] ""Princeton axes matchi...",In article <N4HY.93Apr5120934@harder.ccr-p.ida...
...,...,...,...,...,...,...
18789,18789,talk.religion.misc,84564,sbuckley@fraser.sfu.ca (Stephen Buckley),Re: Religion and marriage,pboxrud@magnus.acs.ohio-state.edu (Paul D Boxr...
18790,18790,talk.religion.misc,84565,bakerj@gtephx.UUCP (Jon Baker),Re: A Message for you Mr. President: How do yo...,"In article <1993Apr23.111105.7703@ifi.uio.no>,..."
18791,18791,talk.religion.misc,84568,pharvey@quack.kfu.com (Paul Harvey),Re: Why did they behave as they did (Waco--rea...,In article <1rc1f3INN7rl@emx.cc.utexas.edu> bi...
18792,18792,talk.religion.misc,84569,<KEVXU@CUNYVM.BITNET>,Re: Info about New Age!,In article <1993Apr26.231845.13843@digi.lonest...


In [5]:
df_20N.head()

,Unnamed: 0,category,email_id,from,subject,body
0,0,alt.atheism,49960,mathew <mathew@mantis.co.uk>,Alt.Atheism FAQ: Atheist Resources,Archive-name: atheism/resources Alt-atheism-ar...
1,1,alt.atheism,51060,mathew <mathew@mantis.co.uk>,Alt.Atheism FAQ: Introduction to Atheism,Archive-name: atheism/introduction Alt-atheism...
2,2,alt.atheism,51119,I3150101@dbstu1.rz.tu-bs.de (Benedikt Rosenau),Re: Gospel Dating,In article <65974@mimsy.umd.edu> mangoe@cs.umd...
3,3,alt.atheism,51120,mathew <mathew@mantis.co.uk>,Re: university violating separation of church/...,dmn@kepler.unh.edu (...until kings become phil...
4,4,alt.atheism,51121,strom@Watson.Ibm.Com (Rob Strom),"Re: [soc.motss, et al.] ""Princeton axes matchi...",In article <N4HY.93Apr5120934@harder.ccr-p.ida...


In [6]:
df_20N.dtypes

Unnamed: 0     int64
category      object
email_id       int64
from          object
subject       object
body          object
dtype: object

In [7]:
def replace_email(dataFrame, category):
    """
    Replaces all emails that contain an "@" and replaces them with "EMAIL".
    """
    # Evitar el warning SettingWithCopy haciendo una copia del DataFrame
    dataFrame = dataFrame.copy()
    
    # Trabajamos sobre la columna específica
    column_data = dataFrame[category].copy()

    for i in range(len(column_data)):
        text = column_data[i]
        
        # Dividimos el texto en palabras
        words = text.split()
        
        for j in words:
            if "@" in j:
                # Encontramos la palabra completa y la reemplazamos por "EMAIL"
                text = text.replace(j, "EMAIL")
        
        # Guardamos el texto actualizado
        column_data[i] = text

    return column_data

In [8]:
def add_spaces_around_special_chars(text):
    """
    Adds spaces around '-' and '/' in the text.
    """
    # Usamos re.sub para añadir espacios alrededor de '-' y '/'
    text = re.sub(r'([-/_.:])', r' \1 ', text)
    
    # Reemplazamos múltiples espacios por un solo espacio
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [9]:
df_20N["from"] = replace_email(df_20N,"from").apply(add_spaces_around_special_chars)
df_20N["body"] = replace_email(df_20N,"body").apply(add_spaces_around_special_chars)
df_20N["subject"] = replace_email(df_20N,"subject").apply(add_spaces_around_special_chars)



In [10]:
df_20N

,Unnamed: 0,category,email_id,from,subject,body
0,0,alt.atheism,49960,mathew EMAIL,Alt . Atheism FAQ : Atheist Resources,Archive - name : atheism / resources Alt - ath...
1,1,alt.atheism,51060,mathew EMAIL,Alt . Atheism FAQ : Introduction to Atheism,Archive - name : atheism / introduction Alt - ...
2,2,alt.atheism,51119,EMAIL (Benedikt Rosenau),Re : Gospel Dating,In article EMAIL EMAIL (Charley Wingate) write...
3,3,alt.atheism,51120,mathew EMAIL,Re : university violating separation of church...,EMAIL ( . . . until kings become philosophers ...
4,4,alt.atheism,51121,EMAIL (Rob Strom),"Re : [soc . motss, et al . ] ""Princeton axes m...",In article EMAIL EMAIL (Bob McGwier) writes : ...
...,...,...,...,...,...,...
18789,18789,talk.religion.misc,84564,EMAIL (Stephen Buckley),Re : Religion and marriage,EMAIL (Paul D Boxrud) writes : > I wasn't sure...
18790,18790,talk.religion.misc,84565,EMAIL (Jon Baker),Re : A Message for you Mr . President : How do...,In article EMAIL EMAIL (Joakim Ruud) writes : ...
18791,18791,talk.religion.misc,84568,EMAIL (Paul Harvey),Re : Why did they behave as they did (Waco - -...,In article EMAIL EMAIL (Bill Jefferys) writes ...
18792,18792,talk.religion.misc,84569,EMAIL,Re : Info about New Age!,In article EMAIL EMAIL (Gerry Palo) >The dange...


In [11]:
df_20N["from"] = df_20N["from"].apply(lambda x: processor_.preprocessing_pipeline(x))
df_20N["body"] = df_20N["body"].apply(lambda x: processor_.preprocessing_pipeline(x))
df_20N["subject"] = df_20N["subject"].apply(lambda x: processor_.preprocessing_pipeline(x))


In [12]:
df_20N

,Unnamed: 0,category,email_id,from,subject,body
0,0,alt.atheism,49960,mathew email,alt atheism faq atheist resourc,archiv name atheism resourc alt atheism archiv...
1,1,alt.atheism,51060,mathew email,alt atheism faq introduct atheism,archiv name atheism introduct alt atheism arch...
2,2,alt.atheism,51119,email benedikt rosenau,gospel date,articl email email charley wingat write well j...
3,3,alt.atheism,51120,mathew email,univers violat separ church state,email king becom philosoph philosoph becom kin...
4,4,alt.atheism,51121,email rob strom,soc motss et al princeton axe match fund boy s...,articl email email bob mcgwier write 1 howev h...
...,...,...,...,...,...,...
18789,18789,talk.religion.misc,84564,email stephen buckley,religion marriag,email paul boxrud write nt sure right newsgrou...
18790,18790,talk.religion.misc,84565,email jon baker,messag mr presid know happen,articl email email joakim ruud write articl em...
18791,18791,talk.religion.misc,84568,email paul harvey,behav waco read suggest,articl email email bill jefferi write would li...
18792,18792,talk.religion.misc,84569,email,info new age,articl email email gerri palo danger anti cult...


# Dividir datos de entrenamiento

In [13]:
df_20N['text'] = df_20N.apply(lambda row: f"{row['from']} {row['subject']} {row['body']}", axis=1)
df_20N.drop(columns=['from','subject','body','email_id'],inplace=True)
# Verifica los resultados


In [14]:
df_20N.columns

Index(['Unnamed: 0', 'category', 'text'], dtype='object')

In [15]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


count vectorizer

In [49]:
texto_procesado = df_20N["text"].tolist()

In [50]:
vectorizer = CountVectorizer(max_features=4000, stop_words=stopwords.words('english'))
texto_features_vectorizer = vectorizer.fit_transform(texto_procesado).toarray()

In [51]:
vectorizer.get_feature_names_out()

array(['00', '000', '01', ..., 'zionist', 'zip', 'zone'], dtype=object)

In [52]:
len(vectorizer.get_feature_names_out().tolist())

4000

In [53]:
texto_procesado[2895].split()[5]

'articl'

tfidf

In [54]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words=stopwords.words('english'))
texto_features_tfidf_vectorizer = tfidf_vectorizer.fit_transform(texto_procesado).toarray()


In [55]:
len(tfidf_vectorizer.get_feature_names_out().tolist())

5000

In [56]:
labels = df_20N['category'].values

In [57]:
# Aca dividimos nuestro dataset en entrenamiento y texto. 20% para evaluar (test) y 80% para entrenamiento (train).
# En las variables X_ quedaran almacenados las presentaciones vectoriales de las críticas.
# En las variables y_ las etiquetas o categorias (polaridad de la crítica).
X_train_v, X_test_v, y_train_v, y_test_v = train_test_split(texto_features_vectorizer, labels, test_size=0.3, random_state=0)

X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(texto_features_tfidf_vectorizer, labels, test_size=0.3, random_state=0)

# Naive Bayes Evaluation

count vectorizer

In [58]:
from sklearn.model_selection import GridSearchCV

In [59]:
nb = MultinomialNB()
# El método fit en sklearn permite ejecutar el proceso de entrenamiento.
nb.fit(X_train_v, y_train_v)

MultinomialNB()

In [60]:
predictions = nb.predict(X_test_v)

In [61]:
print(accuracy_score(y_test_v, predictions))

0.8049299521191701


TF

In [62]:
nb_tf = MultinomialNB()
# El método fit en sklearn permite ejecutar el proceso de entrenamiento.
nb_tf.fit(X_train_tf, y_train_tf)

MultinomialNB()

In [64]:
predictions_tf = nb_tf.predict(X_test_tf)

In [65]:
print(accuracy_score(y_test_tf, predictions))

0.8049299521191701


# Regresion Logistica

In [66]:
from sklearn.linear_model import LogisticRegression

count vectorizer

In [67]:
reg = LogisticRegression()

reg.fit(X_train_v,y_train_v)

C:\Users\Rog\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [68]:
predictions_reg_v = reg.predict(X_test_v)

In [69]:
print(accuracy_score(y_test_v, predictions_reg_v))

0.8489093810959389


tf

In [70]:
reg_tf = LogisticRegression()

reg_tf.fit(X_train_tf,y_train_tf)

LogisticRegression()

In [72]:
predictions_reg_tf = reg_tf.predict(X_test_tf)

In [73]:
print(accuracy_score(y_test_tf, predictions_reg_tf))

0.875687178577762
